# Assignment Group 4
Name: Akhilesh Yadav Gaddam

## Module A _(54 points)_
For this assignment your work will set up a toolkit for plain text enhancement and summarization for reuse later. Specifically, you'll use an API service known as Wikifier to embed hyperlink references to encyclopedia articles.

__A1.__ _(2 points)_ Throughout this problem, we're going to be working with the Wikifier API, which is "a web service that takes a text document as input and annotates it with links to relevant Wikipedia concepts". In other words, by using this service, you can embed links to Wikipedia articles in the original text. So, before we can begin with the actual work, it's necessary to request access to this API at the following link: http://wikifier.org/info.html. Read this documentation and familiarize yourself with it, and make sure to register for a `userKey`.
Store your `userKey` in the below cell:

In [1]:
key="eznwtjcjgtshocxtktzfmbtuaoeqlm"

__A2.__ _(7 points)_ We're going to make requests to Wikifier. For this, create a function that takes a block of text and `userKey` as input, and outputs the JSON response given by the Wikification API (per the following specifications) interpreted as a Python object (deserialized). __Important__: For full credit you must read the Wikifier docs: http://wikifier.org/info.html; specifically, you must determine how to build a URL that sets the `'applyPageRankSqThreshold'` field to `True`, the corresponding `'pageRankSqThreshold'` field to `0.8`, and the language field to `'en'`. Please find a sample output in the below. The input text was `"Drexel University"`.

In [2]:
import urllib.parse, urllib.request, json


def wikify(text,key):
    threshold=0.8
    data = urllib.parse.urlencode([
        ("text", text), ("lang", "en"),
        ("applyPageRankSqThreshold","True"),("pageRankSqThreshold","%g" % threshold),
        ("userKey", key),
        ("wikiDataClasses", "True"), ("wikiDataClassIds", "False")
        ])
    url = 'http://www.wikifier.org/annotate-article'
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    return response

In [3]:
wikify("Drexel University","eznwtjcjgtshocxtktzfmbtuaoeqlm")

{'annotations': [{'title': 'Drexel University',
   'url': 'http://en.wikipedia.org/wiki/Drexel_University',
   'lang': 'en',
   'pageRank': 0.1334028435275827,
   'cosine': 0.7571123987050522,
   'secLang': 'en',
   'secTitle': 'Drexel University',
   'secUrl': 'http://en.wikipedia.org/wiki/Drexel_University',
   'wikiDataItemId': 'Q603034',
   'wikiDataClasses': [{'itemId': 'Q15936437',
     'enLabel': 'research university'},
    {'itemId': 'Q23002054',
     'enLabel': 'private not-for-profit educational institution'},
    {'itemId': 'Q3918', 'enLabel': 'university'},
    {'itemId': 'Q23002042', 'enLabel': 'private educational institution'},
    {'itemId': 'Q163740', 'enLabel': 'nonprofit organization'},
    {'itemId': 'Q38723', 'enLabel': 'higher education institution'},
    {'itemId': 'Q4671277', 'enLabel': 'academic institution'},
    {'itemId': 'Q2385804', 'enLabel': 'educational institution'},
    {'itemId': 'Q43229', 'enLabel': 'organization'},
    {'itemId': 'Q155076', 'enLabel

__A3.__ _(4 points)_ Check to make sure your function works! Load the small block of text provided under `./data/texts/example_text.txt`, and apply your `Wikify()` function. Print out the result. Does everything look correct, according to the docs? Discuss the output in the response box.

_Response_.

The output isn't in the format specified in documentation.

In [4]:
with open('data/texts/example_text.txt', 'r') as f:
    file=f.read().split('\n')

content = ' '.join(map(str, file))

wikify(content,"eznwtjcjgtshocxtktzfmbtuaoeqlm")

{'annotations': [{'title': 'Hey (band)',
   'url': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'lang': 'en',
   'pageRank': 0.001979565907847362,
   'cosine': 0.07484430544881877,
   'secLang': 'en',
   'secTitle': 'Hey (band)',
   'secUrl': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'wikiDataItemId': 'Q2632769',
   'wikiDataClasses': [{'itemId': 'Q215380', 'enLabel': 'band'},
    {'itemId': 'Q2088357', 'enLabel': 'musical ensemble'},
    {'itemId': 'Q32178211', 'enLabel': 'music organisation'},
    {'itemId': 'Q16334295', 'enLabel': 'group of humans'},
    {'itemId': 'Q43229', 'enLabel': 'organization'},
    {'itemId': 'Q16334298', 'enLabel': 'group of living things'},
    {'itemId': 'Q24229398', 'enLabel': 'agent'},
    {'itemId': 'Q61961344', 'enLabel': 'group of physical objects'},
    {'itemId': 'Q517966', 'enLabel': 'structure'},
    {'itemId': 'Q23958946', 'enLabel': 'individual'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q35120', 'enLabel': 'entity'

__A4.__ (7 pts) Now we want to make our code reusable, and don't want to cause any server problems for wikifier.org. In other words, we want to locally store data that has already been requested and processed and try to make as few requests to the server as possible. So, write a function called `get_wikification(filename, KEY)` that accepts a `filename` of a text file (assuming a `'.txt'` extension) from `'./data/texts/'` and Wikifier `userKey` and does the following:

1. Checks to see if a corresponding filename (with `'.json'` extension) exists in the `'./data/wikifications/'` directory, and
    1. loads the text's wikification from the json file (only if it exists!!) or
    - requests the text's wikification from wikifier.org (only if the file from __1.A__ doesn't exist!) and
        1. stores the newly received wikification in `'data/wikifications/'` under the corresponding file name,
- but always `return`s the text's wikifications to the user.

\[__Hint.__ For __1.A__ vs. __1.B__ in the above steps, you can check to see if a file exists using the `os` module's
`os.path.exists(file_path)` function.\]


In [5]:
import re, os
def get_wikification(filename, KEY):
    wikification_name = filename
    if os.path.exists('./data/wikifications/'+wikification_name):
        wikification = json.load(open('./data/wikifications/'+wikification_name, 'r'))
    else:
        wikification = wikify(open("./data/texts/"+filename, "r").read(), KEY)
        with open('./data/wikifications/'+wikification_name, 'w') as f: 
            f.write(json.dumps(wikification))
    return(wikification)

In [6]:
get_wikification('example_text.txt','eznwtjcjgtshocxtktzfmbtuaoeqlm')

{'annotations': [{'title': 'Hey (band)',
   'url': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'lang': 'en',
   'pageRank': 0.001979565907847362,
   'cosine': 0.07484430544881877,
   'secLang': 'en',
   'secTitle': 'Hey (band)',
   'secUrl': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'wikiDataItemId': 'Q2632769',
   'wikiDataClasses': [{'itemId': 'Q215380', 'enLabel': 'band'},
    {'itemId': 'Q2088357', 'enLabel': 'musical ensemble'},
    {'itemId': 'Q32178211', 'enLabel': 'music organisation'},
    {'itemId': 'Q16334295', 'enLabel': 'group of humans'},
    {'itemId': 'Q43229', 'enLabel': 'organization'},
    {'itemId': 'Q16334298', 'enLabel': 'group of living things'},
    {'itemId': 'Q24229398', 'enLabel': 'agent'},
    {'itemId': 'Q61961344', 'enLabel': 'group of physical objects'},
    {'itemId': 'Q517966', 'enLabel': 'structure'},
    {'itemId': 'Q23958946', 'enLabel': 'individual'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q35120', 'enLabel': 'entity'

__A5.__ _(6 points)_ Now, to work with the Wikifier output we'll need to go through a few steps. Annotation (embedded links) provided by Wikifer reference specific document indices, i.e., for words (of their tokenization) and characters. We'll work on the character level because it's more precise. To get this off of the ground, make a function called `build_doc(wiki)`, that takes a Wikifier output, `wiki`, and builds up the original document from the `wiki['words']` and `wiki['spaces']` fields. 

\[__Hint.__ Wikifier tokenized text and store words in the `wiki['words'] field and white spaces in the `wiki['spaces'] field. These two fields alternate,  with `wiki['spaces']` having exactly one more element than `wiki['words']`, i.e., a rebuilt document will join the elements of the two with the first element coming from `wiki['spaces']`\].

In [7]:
output=wikify(content,'eznwtjcjgtshocxtktzfmbtuaoeqlm')

def build_doc(wiki):
    words=wiki["words"]
    spaces=wiki["spaces"]
    doc=spaces[0]
    for i in range(0,len(spaces[1:])):
        add= words[i]+spaces[i+1]
        doc=doc+add
    return doc

In [8]:
build_doc(output)

"Hey smilin' strange You're lookin' happily deranged Could you settle to shoot me? Or have you picked your target yet? Hey Sandy Don't you talk back, Hey Sandy Four feet away End of speech, it's the end of the day We was only funnin' But guiltily I thought you had it comin' "

__A6.__ _(6 points)_ Now that the document is back together we want to collect the links from the annotations. Write a function called `get_links(wiki)` that processes the `annotation` objects from the `wiki['annotations']` field (a list of `annotation` objects), and creates/outputs a data object called links, which is a list of tuples:
```
links = [(chFrom, chTo, hyperlink), ...]
```
Hyperlinks will come from `annotation["url"]`, and the `chFrom` and `chTo` elements represent character ranges corresponding to embeddings of `annotation["url"]` in the document. Note that there are multiple character ranges: one for each `support` object in `annotation["support"]` (also a list).  Specifically, each `support` object in the annotation field has two character indices keyed as `support["chFrom"]` and `support["chTo"]` which designate a range (in character indices) over which the hyperlink might well be embedded. In many cases, two character indices are identical which means the hyperlink will be embedded over one character. 

In [9]:
def get_links(wiki):
    links=[]
    for i in wiki["annotations"]:
        for j in i['support']:
            temp=[]
            temp.append(j['chFrom'])
            temp.append(j['chTo'])
            temp.append(i['url'])
            
            links.append(temp)
    return links

In [10]:
get_links(output)

[[0, 2, 'http://en.wikipedia.org/wiki/Hey_(band)'],
 [118, 120, 'http://en.wikipedia.org/wiki/Hey_(band)'],
 [149, 151, 'http://en.wikipedia.org/wiki/Hey_(band)'],
 [10, 10, 'http://en.wikipedia.org/wiki/Foot_(unit)'],
 [33, 33, 'http://en.wikipedia.org/wiki/Foot_(unit)'],
 [164, 167, 'http://en.wikipedia.org/wiki/Foot_(unit)'],
 [231, 231, 'http://en.wikipedia.org/wiki/Foot_(unit)'],
 [272, 272, 'http://en.wikipedia.org/wiki/Foot_(unit)'],
 [12, 18, 'http://en.wikipedia.org/wiki/Strange_quark'],
 [20, 25, 'http://en.wikipedia.org/wiki/You'],
 [35, 41, 'http://en.wikipedia.org/wiki/Happiness'],
 [43, 50, 'http://en.wikipedia.org/wiki/Mental_disorder'],
 [52, 56, 'http://en.wikipedia.org/wiki/English_modal_verbs'],
 [58, 60, 'http://en.wikipedia.org/wiki/Personal_pronoun'],
 [90, 92, 'http://en.wikipedia.org/wiki/Personal_pronoun'],
 [134, 136, 'http://en.wikipedia.org/wiki/Personal_pronoun'],
 [256, 258, 'http://en.wikipedia.org/wiki/Personal_pronoun'],
 [62, 67, 'http://en.wikipedia.o

__A7.__ _(5 points)_ Your next job is to create an `embed_link(doc, link)` function that takes the full document output from __A4__, and, a given link (list element) from __A6__'s output, and returns an updated document with hyperlink embedded. Exhibit the function of `embed_link` using `IPython`'s `HTML()` function: `from IPython.core.display import HTML`

In [11]:
from IPython.core.display import HTML
link= get_links(output)
doc=build_doc(output)
link1=link[0]
def embed_link(doc, link):
    sentence=''
    for i in link:
        show_text = doc[i[0]:i[1]+1]
        annotated_doc = ( 
                     "<a href=\"" + i[2] + "\">" + show_text + "</a>")
        if annotated_doc not in sentence:
            
            sentence=sentence +' '+ annotated_doc
    return(sentence)

In [12]:
a=embed_link(doc, link)
HTML(a)

__A8.__ _(2 points)_ The responses received from Wikifier are really only _predictions_ of where links should go. This means that multiple link predictions might span overlapping portions of source text. So, we'll have to make sure our code does some light decision making to avoid overlapping hyperlinks and broken html code. Review the Wikifier response data and discuss any viable approach to dealing with the possible overlapping link problem in the response box below. \[__Hint__: If one such approach is taken in problem __A7__ that you may discuss here. If you do so, you are required to provide a detailed description of _how_ __A7__ resolves the overlapping-link problem and any limitations of its approach.\]

__Note__: Even if you have not yet generated any Wikifier output through Problem A, you may complete this part for full credit using the sample data provided in `./data/example_text.json`.

_Response_.

Looking at A7 output we can notice overlap of the link with source text. We can sort the data. If the last value of first link index is less than start value of next link, if the value is not greater than previous value, we can ignore.

__A9.__ _(8 points)_ Here is where we're going to build a wrapper function, called `embed_links()`, to manage the link embedding process, being sure to avoid the overlapping link problem. Specifically, your function _must_ do the following, in order:

1. Take a wikifier, `wiki`, response as input.
2. Build the document using `build_doc()` and store its output as a string called `doc`
3. Apply `get_links()` function, storing the `links` output 
4. _Reverse_ sort the `link`  by their ending points, i.e., `support["chTo"]` values
5. Loop through the _sorted_ `links` 
6. Conditionally use `embed_link()` function to embed links into `doc` in the loop. Specifically, only embed a link if its `support["chTo"]` value is less than the previous one's `support["chFrom"]` value, or the link to embed is the first in the loop.
7. `return` the link-enhanced doc and exhibit it's output using `HTML()`.

__Note__: If you have not yet completed __Problem A4__, you can complete this part for full credit using the sample file, `./data/example_text.json`. This file has the correct input format of data for your function (a Wikifier json output).

In [13]:
def embed_links(wiki):
    doc = build_doc(wiki)
    links = get_links(wiki)
    links = sorted(links, key = lambda x: x[1], reverse = True)
    sentence=''
    for i in links:
        show_text = doc[i[0]:i[1]+1]
        annotated_doc = ( 
                     "<a href=\"" + i[2] + "\">" + show_text + "</a>")
        if annotated_doc not in sentence:
            
            sentence=sentence +' '+ annotated_doc
    return(sentence)

In [14]:
a=embed_links(output)
HTML(a)

__A10.__ _(2 points)_ The `embed_links()` function's design overcomes an additional challenge. Your job here is to explain in detail how it does so. Here's the challenge: since we are using character indices to insert the links over ranges of the input document, adding the markup required for html _necessarily_ creates a longer document and throws off the original document's character indices from their original positions. How does __A9__'s design avoid this _index modification_ problem and simply utilize the exact character indices provided by Wikifier?

_Response_.

We will check indices in the reference document and causing changes in the first doc via searching for the word and replacing it. In this way, B5's design avoids this problem and simply utilize the indices provided by Wikifier.

__A11.__ _(5 points)_ It's time to package your code up so someone else can use it! Specifically, the goal here is to place _all of the functions_ defined within this module in a file called `./wikifier.py`. Doing so will technically create our very own _Python module_!

When this is complete, exhibit your code's function by loading the module, i.e., `import wikifier`, and executing the two controller methods:

- `wikification = wikifier.get_wikification("example_text.txt", USER_KEY)`
- `annotated_doc = wikifier.embed_links(wikification)`

Note: Your `./wikifier.py` Python must also contain any necessary module loads, and since it is being distributed for _others_ to use it absolutely must use relative file paths, assuming the `./data/texts/` and `./data/wikifications/` file structure. So if these items aren't in place in your functions above, you must modify your code!

\[Hint. to write the contents of a cell to a file (instead of executing in the ipython kernel), place the following magic command at the top of the cell: `%%writefile filename.extension`\]

In [15]:
%%writefile wikifier1.py

import urllib.parse, urllib.request, json, os

from IPython.core.display import HTML

def wikify(text,key):
    threshold=0.8
    data = urllib.parse.urlencode([
        ("text", text), ("lang", "en"),
        ("applyPageRankSqThreshold","True"),("pageRankSqThreshold","%g" % threshold),
        ("userKey", key),
        ("wikiDataClasses", "True"), ("wikiDataClassIds", "False")
        ])
    url = 'http://www.wikifier.org/annotate-article'
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    return response


def get_wikification(filename, KEY):
    wikification_name = filename
    if os.path.exists('./data/wikifications/'+wikification_name):
        wikification = json.load(open('./data/wikifications/'+wikification_name, 'r'))
    else:
        wikification = wikify(open("./data/texts/"+filename, "r").read(), KEY)
        with open('./data/wikifications/'+wikification_name, 'w') as f: 
            f.write(json.dumps(wikification))
    return(wikification)



def build_doc(wiki):
    words=wiki["words"]
    spaces=wiki["spaces"]
    doc=spaces[0]
    for i in range(0,len(spaces[1:])):
        add= words[i]+spaces[i+1]
        doc=doc+add
    return doc


def get_links(wiki):
    links=[]
    for i in wiki["annotations"]:
        for j in i['support']:
            temp=[]
            temp.append(j['chFrom'])
            temp.append(j['chTo'])
            temp.append(i['url'])
            
            links.append(temp)
    return links


def embed_link(doc, link):
    sentence=''
    for i in link:
        show_text = doc[i[0]:i[1]+1]
        annotated_doc = ( 
                     "<a href=\"" + i[2] + "\">" + show_text + "</a>")
        if annotated_doc not in sentence:
            
            sentence=sentence +' '+ annotated_doc
    return(sentence)


def embed_links(wiki):
    doc = build_doc(wiki)
    links = get_links(wiki)
    links = sorted(links, key = lambda x: x[1], reverse = True)
    sentence=''
    for i in links:
        show_text = doc[i[0]:i[1]+1]
        annotated_doc = ( 
                     "<a href=\"" + i[2] + "\">" + show_text + "</a>")
        if annotated_doc not in sentence:
            
            sentence=sentence +' '+ annotated_doc
    return(sentence)

Overwriting wikifier1.py


In [16]:
import wikifier1
wikification = wikifier1.get_wikification("example_text.txt","eznwtjcjgtshocxtktzfmbtuaoeqlm")
wikification

{'annotations': [{'title': 'Hey (band)',
   'url': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'lang': 'en',
   'pageRank': 0.001979565907847362,
   'cosine': 0.07484430544881877,
   'secLang': 'en',
   'secTitle': 'Hey (band)',
   'secUrl': 'http://en.wikipedia.org/wiki/Hey_(band)',
   'wikiDataItemId': 'Q2632769',
   'wikiDataClasses': [{'itemId': 'Q215380', 'enLabel': 'band'},
    {'itemId': 'Q2088357', 'enLabel': 'musical ensemble'},
    {'itemId': 'Q32178211', 'enLabel': 'music organisation'},
    {'itemId': 'Q16334295', 'enLabel': 'group of humans'},
    {'itemId': 'Q43229', 'enLabel': 'organization'},
    {'itemId': 'Q16334298', 'enLabel': 'group of living things'},
    {'itemId': 'Q24229398', 'enLabel': 'agent'},
    {'itemId': 'Q61961344', 'enLabel': 'group of physical objects'},
    {'itemId': 'Q517966', 'enLabel': 'structure'},
    {'itemId': 'Q23958946', 'enLabel': 'individual'},
    {'itemId': 'Q16887380', 'enLabel': 'group'},
    {'itemId': 'Q35120', 'enLabel': 'entity'

In [17]:
annotated_doc = wikifier1.embed_links(wikification)

In [18]:
HTML(annotated_doc)